# PyTorch：view() 与 reshape() 区别详解
<font size=4 color=red>
    总之，两者都是用来重塑tensor的shape的。view只适合对满足连续性条件（contiguous）的tensor进行操作，而reshape同时还可以对不满足连续性条件的tensor进行操作，具有更好的鲁棒性。view能干的reshape都能干，如果view不能干就可以用reshape来处理。别看目录挺多，但内容很细呀~其实原理并不难啦~我们开始吧~ 
</font></br>



## 主要内容

### 一、PyTorch中tensor的存储方式

    1、PyTorch张量存储的底层原理

    2、PyTorch张量的步长（stride）属性

### 二、对“视图(view)”字眼的理解

### 三、view() 和reshape() 的比较

    1、对 torch.Tensor.view() 的理解

    2、对 torch.reshape() 的理解

### 四、总结

### 一、PyTorch中tensor的存储方式
<font size=4>想要深入理解view与reshape的区别，首先要理解一些有关PyTorch张量存储的底层原理，比如tensor的头信息区（Tensor）和存储区 （Storage）以及tensor的步长Stride。不用慌，这部分的原理其实很简单的(^-^)!</font>

<font size=5>1、PyTorch张量存储的底层原理</font></br>
<font size=4>tensor数据采用头信息区（Tensor）和存储区 （Storage）分开存储的形式，如图所示。变量名以及其存储的数据是分为两个区域分别存储的。比如，我们定义并初始化一个tensor，tensor名为A，A的形状size、步长stride、数据的索引等信息都存储在头信息区（Tensor），而A所存储的真实数据则存储在存储区（Storage）。另外，如果我们对A进行截取、转置或修改等操作后赋值给B，则B的数据共享A的存储区（Storage），存储区（Storage）的数据数量没变，变化的只是B的头信息区（Tensor）对数据的索引方式。</font>
</br>
<img src="https://github.com/jhong-tao/s2dl/blob/master/ipynb/0-imgs/217856618_1_20210317091321662.jpg?raw=true" width=500 >
</br>（Tensor）


## 共用存储区域举例
<font size=4>共用存储区域，当一个变量修改存储区的数据时，另一个变量的值也会跟着变化

In [2]:
import torch
a = torch.arange(5)  # 初始化张量 a 为 [0, 1, 2, 3, 4]
b = a[2:]            # 截取张量a的部分值并赋值给b，b其实只是改变了a对数据的索引方式
print('a:', a)
print('b:', b)
print('id of storage of a:', id(a.storage))  # 打印a的存储区地址
print('id of storage of b:', id(b.storage))  # 打印b的存储区地址,可以发现两者是共用存储区

a: tensor([0, 1, 2, 3, 4])
b: tensor([2, 3, 4])
id of storage of a: 2175559017080
id of storage of b: 2175559017080


In [4]:
b[1] = 0    # 修改b中索引为1，即a中索引为3的数据为0
print('a:', a)
print('b:', b)
print('id of storage of a:', id(a.storage))  # 打印a的存储区地址,可以发现a的相应位置的值也跟着改变，说明两者是共用存储区
print('id of storage of b:', id(b.storage))  # 打印b的存储区地址

a: tensor([0, 1, 2, 0, 4])
b: tensor([2, 0, 4])
id of storage of a: 2175478696680
id of storage of b: 2175478696680


### PyTorch张量的步长（stride）属性
<font size=4> 
torch的tensor也是有步长（stride）属性的，说起stride属性是不是很耳熟？是的，卷积神经网络中卷积核对特征图的卷积操作也是有stride属性的，但这两个stride可完全不是一个意思哦。
</font>
<br>
<font size=4 color=red>
stride是在指定维度dim中从一个元素跳到下一个元素所必需的步长。当没有参数传入时，返回所有步长的元组。否则，将返回一个整数值作为特定维度dim中的步长。<br>
tensor的步长可以理解为从指定的维度中的一个元素到下一个元素中间的跨度。<br>为方便理解，就直接用图说明了，您细细品(^-^)：    
</font>
<img src="">